In [1]:
import sys
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')

import glob, os
import pandas as pd
import json
import spacy
import nltk
import numpy as np
import json
#import smogn
import seaborn as sns
import pickle
from bs4 import BeautifulSoup
from pysbd.utils import PySBDFactory
import math

from sumeval.metrics.rouge import RougeCalculator
rouge = RougeCalculator(stopwords=True, lang="en")
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from timeit import default_timer as timer 
from imblearn.over_sampling import SMOTE
from sklearn.metrics import matthews_corrcoef
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from imblearn.under_sampling import RandomUnderSampler
#from tensorflow.keras.models import model_from_json

#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('punkt')
#nltk.download('stopwords')


#import en_core_web_sm
#nlp_md = en_core_web_sm.load()

#import en_core_web_md
#nlp_md = en_core_web_md.load()

#!python -m spacy download en_core_web_sm
nlp_sm = spacy.load('en_core_web_sm')

#import en_core_web_sm
#nlp_md = en_core_web_sm.load()y

#import en_core_web_md
#nlp_md = en_core_web_md.load()
#!python -m spacy download en_core_web_md
nlp_md = spacy.load('en_core_web_md')

import warnings
warnings.filterwarnings("ignore")

path_base = "/scratch/cinthiasouza/mv-text-summarizer"
path_to_read="/scratch/cinthiasouza/mv-text-summarizer/result_/{}/{}_*.csv"


In [2]:
%load_ext autoreload
%autoreload 2

from src import preprocess
from src import extract_features
from src import tokenizer
from src import create_features_df
from src import transform_data
from src import loader
from src import utils
from src import ensemble_tree_models
from src import tunning_hyperparametrs as th
#from src import mlp_regressor
#from src import mlp_classifier
from src import summarization
from src import normalization
from src import ensemble_tree_models as classifiers
from src import utils_classification as utils_clf
from src import evaluate_classifiers as ev
from src import prepare_data
from src import display_results as dr
import joblib
from joblib import Parallel, delayed
#from tensorflow.keras.utils import to_categorical
#from src import pipeline_extract_features as pef

In [3]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Dropout, InputLayer, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.callbacks import Callback

In [4]:
with open('dataset6_{}.pkl'.format('features'), 'rb') as fp:
    dataset = pickle.load(fp)

columns_name = ['text_rank', 'lex_rank', 'count_one_gram', 'count_two_gram',
       'count_three_gram', 'count_article_keywords', 'tf-isf', 'pos_score', 'ner_score']

sections=['introduction', 'materials', 'conclusion']

folder_to_save = 'dataset6'
path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/{}".format(folder_to_save)

# Spacy

In [5]:
def create_embeddings(sentence):
    
    try:
        doc = nlp_md(sentence)
        embeddings = doc.vector
    except TypeError:
        embeddings = [0] *300
    return embeddings

In [6]:
train_columns = ['text_rank', 'lex_rank', 'count_one_gram', 'count_two_gram',
       'count_three_gram', 'count_article_keywords',
       'tf-isf', 'position_score', 'paragraph_score',
       'length_score', 'pos_score', 'ner_score', 'dist_centroid']

In [101]:
section = "introduction"

features = dataset[section][4][['sentences', 'articles', 'rouge_1', 'bin']]
features = features.reset_index(drop=True)

X_train, y_train = RandomUnderSampler(random_state=47).fit_resample(features[['sentences', 'articles', 'rouge_1']],
                         features['bin'])

embeddings = [create_embeddings(row['sentences']) for idx, row in X_train.iterrows()]
df = pd.DataFrame(embeddings)
df['label'] = y_train

df.to_csv("dataset6/embed_{}_train.csv".format (section), index=False)

In [117]:
section = "materials"

features = dataset[section][4][['sentences', 'articles', 'rouge_1', 'bin']]
features = features.reset_index(drop=True)

X_train, y_train = RandomUnderSampler(random_state=47).fit_resample(
    features[['sentences', 'articles', 'rouge_1']],features['bin'])

embeddings = [create_embeddings(row['sentences']) for idx, row in X_train.iterrows()]
df = pd.DataFrame(embeddings)
df['label'] = y_train

df.to_csv("{}/embed_{}_train.csv".format (path_to_save, section), index=False)

In [118]:
section = "conclusion"

features = dataset[section][4][['sentences', 'articles', 'rouge_1', 'bin']]
features = features.reset_index(drop=True)

X_train, y_train = RandomUnderSampler(random_state=47).fit_resample(features[['sentences', 'articles', 'rouge_1']],
                                                     features['bin'])

embeddings = [create_embeddings(row['sentences']) for idx, row in X_train.iterrows()]
df = pd.DataFrame(embeddings)
df['label'] = y_train

df.to_csv("{}/embed_{}_train.csv".format (path_to_save, section), index=False)

In [8]:
section = "introduction"

features = dataset[section][5][['sentences', 'articles', 'rouge_1', 'bin']]
features = features.reset_index(drop=True)

In [12]:
section = "introduction"

features = dataset[section][5][['sentences', 'articles', 'rouge_1', 'bin']]
features = features.reset_index(drop=True)

embeddings = [create_embeddings(row['sentences']) for idx, row in features.iterrows()]
df = pd.DataFrame(embeddings)
df['label'] = features['bin']

df.to_csv("{}/embed_{}_test.csv".format (path_to_save, section), index=False)

In [13]:
section = "materials"

features = dataset[section][5][['sentences', 'articles', 'rouge_1', 'bin']]
features = features.reset_index(drop=True)

embeddings = [create_embeddings(row['sentences']) for idx, row in features.iterrows()]
df = pd.DataFrame(embeddings)
df['label'] = features['bin']

df.to_csv("{}/embed_{}_test.csv".format (path_to_save, section), index=False)

In [14]:
section = "conclusion"

features = dataset[section][5][['sentences', 'articles', 'rouge_1', 'bin']]
features = features.reset_index(drop=True)

embeddings = [create_embeddings(row['sentences']) for idx, row in features.iterrows()]
df = pd.DataFrame(embeddings)
df['label'] = features['bin']

df.to_csv("{}/embed_{}_test.csv".format (path_to_save, section), index=False)

In [62]:
df.to_csv("{}/embed_introduction_train.csv".format(path_to_save), index=False)

# Classificação com embeddings

In [7]:
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [8]:
def remove_unicode(text):

    text = str(text).encode("ascii", "ignore")
    text = text.decode()

    return text

In [9]:
def clean_text(text):
    
    text = re.sub(r"[^a-zA-Z]+", ' ', text)
    text = text.lower()
    return text

In [10]:
def count_words(text):
    
    return len(text.split(" "))

In [11]:
section='conclusion'

with open('dataset6_{}.pkl'.format('features'), 'rb') as fp:
    dataset = pickle.load(fp)

X_features = dataset[section][0]
y_features = dataset[section][2]

columns = list(range(0, 300))
columns = list(map(str, columns))

folder_to_save = 'models_v6'
path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/{}".format(folder_to_save)

X_embedd = pd.read_csv("dataset6/embed_{}_train.csv".format(section))

y_embedd = X_embedd['label']
X_embedd = X_embedd[columns]

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [13]:
one_hot_label = to_categorical(y_embedd)
X_train_embedd, X_valid_embedd, y_train_embedd, y_valid_embedd = train_test_split(
    X_embedd, y_embedd, stratify=one_hot_label, shuffle=True, test_size=0.2)

one_hot_label = to_categorical(y_features)
X_train_features, X_valid_features, y_train_features, y_valid_features = train_test_split(
    X_features, one_hot_label, stratify=one_hot_label, shuffle=True, test_size=0.2)

In [64]:
sentences = sentences.apply(remove_unicode)
sentences = sentences.apply(clean_text)

t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1
print(vocab_size)

(61120, 18)

In [35]:
sentences = dataset[section][5]['sentences']
sentences = sentences.apply(remove_unicode)
sentences = sentences.apply(clean_text)

encoded_docs = t.texts_to_sequences(sentences)
# pad documents to a max length of 4 words
max_length = 300
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

encoded_docs = [one_hot(d, vocab_size) for d in sentences]

max_length = 500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [85]:
padded_docs_train = padded_docs

In [36]:
padded_docs.shape

(61120, 500)

In [79]:
padded_docs_test = padded_docs

In [82]:
padded_docs_test.shape

(61120, 500)

In [17]:
embeddings_index = dict()

f = open(r'../glove.840B.300d.txt', "r", encoding="utf8")
for line in f:
    values = line.split()
    word = ''.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 2195892 word vectors.


In [23]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


IndexError: index 48328 is out of bounds for axis 0 with size 48328

In [24]:
embedding_matrix.shape

(48328, 300)

In [27]:
model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=True)
model.add(e)
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(300, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(padded_docs, y_features, epochs=6, verbose=1)

Epoch 1/6
1919/1919 [==============================] - 848s 441ms/step - loss: 0.6997 - accuracy: 0.5032
Epoch 2/6
1919/1919 [==============================] - 845s 440ms/step - loss: 0.6943 - accuracy: 0.5062
Epoch 3/6
1919/1919 [==============================] - 866s 451ms/step - loss: 0.6879 - accuracy: 0.5427
Epoch 4/6
1919/1919 [==============================] - 851s 443ms/step - loss: 0.5487 - accuracy: 0.7166
Epoch 5/6
1919/1919 [==============================] - 852s 444ms/step - loss: 0.2217 - accuracy: 0.9056
Epoch 6/6
1919/1919 [==============================] - 870s 454ms/step - loss: 0.0776 - accuracy: 0.9709


In [37]:
y_test = dataset[section][5]['bin']
y_test.shape

(61120,)

In [38]:
model.evaluate(padded_docs, y_test)

1910/1910 [==============================] - 295s 154ms/step - loss: 1.9350 - accuracy: 0.5017


[1.9349699020385742, 0.5016524791717529]